In [33]:
%cd PRO504

[Errno 2] No such file or directory: 'PRO504'
/workspace/PRO504


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType,BooleanType
from pyspark.sql import functions as fun


spark = ( SparkSession.builder.appName("PRO502").master("spark://spark-master:7077").getOrCreate())

sc = spark.sparkContext


In [35]:
main_schema = StructType([
    StructField("Place_Name",StringType(),True),
    StructField("Country",StringType(),True),
    StructField("City",StringType(),True),
    StructField("Annual_Visitors_Millions",FloatType(),True),
    StructField("Type",StringType(),True),
    StructField("Unsesco_World_Heritage",StringType(),True),
    StructField("Year_Built",StringType(),True),
    StructField("Entry_Fee_USD",IntegerType(),True),
    StructField("Best_Visit_Month",StringType(),True),
    StructField("Region",StringType(),True),
    StructField("Tourism_Revenue_Million_USD",IntegerType(),True),
    StructField("Average_Visit_Duration_Hours",FloatType(),True),
    StructField("Famous_For",StringType(),True),
])

df = (spark.read.format("csv")
      .option("header","true")
      .schema(main_schema)
      .load("world_famous_places_2024.csv"))

In [36]:
df_feat = df.withColumn(
    "SKU_Lugar",
    fun.concat_ws(
        "_",
        fun.substring(fun.upper(fun.col("Country"),),1,3),
        fun.rpad(fun.col("City"), 3, "X"),
        fun.split(fun.col("Type")," ").getItem(0)
    )
)

df_feat.select("SKU_Lugar").show(25,truncate=False)

+--------------------------+
|SKU_Lugar                 |
+--------------------------+
|FRA_Par_Monument/Tower    |
|UNI_New_Urban             |
|FRA_Par_Museum            |
|CHI_Bei_Historic          |
|IND_Agr_Monument/Mausoleum|
|ITA_Rom_Historic          |
|UNI_New_Monument/Statue   |
|AUS_Syd_Cultural          |
|PER_Cus_Archaeological    |
|CHI_Bei_Historic          |
|CAM_Sie_Temple            |
|FRA_Par_Cathedral         |
|UNI_New_Park              |
|UNI_Las_Entertainment     |
|UNI_New_Skyscraper        |
|UNI_San_Bridge            |
|UNI_Was_Monument/Memorial |
|SPA_Bar_Cathedral         |
|UNI_Ana_Theme             |
|UNI_Orl_Theme             |
|UNI_Ari_Natural           |
|EGY_Cai_Historic          |
|ITA_Pis_Historic          |
|FRA_Ver_Historic          |
|GRE_Ath_Archaeological    |
+--------------------------+
only showing top 25 rows



In [37]:
df_feat = df_feat.withColumn(
    "Duracion_Techo",
    fun.ceil(fun.col("Average_Visit_duration_Hours"))
)

df_feat = df_feat.withColumn(
    "Log_Ingresos",
    fun.log10(fun.col("Tourism_Revenue_Million_USD"))
)
df_feat = df_feat.withColumn(
    "Mejor_Oferta",
    fun.least(fun.col("Entry_Fee_USD"),fun.lit(20))
)

df_feat.select("Duracion_Techo", "Log_Ingresos","Mejor_Oferta").show(25,truncate=False)

+--------------+------------------+------------+
|Duracion_Techo|Log_Ingresos      |Mejor_Oferta|
+--------------+------------------+------------+
|3             |1.9777236052888478|20          |
|2             |1.845098040014257 |0           |
|4             |2.0791812460476247|20          |
|4             |2.255272505103306 |10          |
|2             |1.8129133566428555|15          |
|3             |1.9294189257142929|18          |
|2             |1.6532125137753437|20          |
|2             |2.041392685158225 |20          |
|8             |2.255272505103306 |20          |
|3             |1.8750612633917   |8           |
|6             |1.9542425094393248|20          |
|1             |1.6989700043360187|0           |
|3             |1.5440680443502757|0           |
|8             |3.832508912706236 |0           |
|2             |2.060697840353612 |20          |
|1             |1.9030899869919435|0           |
|1             |1.3979400086720377|0           |
|2             |2.11

In [38]:
df_feat = df_feat.withColumn(
    "Desc_Corta",
    fun.substring(fun.col("Famous_for"),1,15)
)

df_feat = df_feat.withColumn(
    "Ciudad_Limpia",
    fun.regexp_replace(fun.col("City"),"[a-z .]", "")
)


df_feat.select("Desc_Corta","Ciudad_Limpia").show(25,truncate=False)

+---------------+-------------+
|Desc_Corta     |Ciudad_Limpia|
+---------------+-------------+
|Iconic iron lat|P            |
|Bright lights, |NYC          |
|World's most vi|P            |
|Ancient defensi|B/M          |
|White marble ma|A            |
|Ancient Roman a|R            |
|Symbol of freed|NYC          |
|Unique sail-lik|S            |
|Ancient Incan c|CR           |
|Imperial palace|B            |
|Largest religio|SR           |
|Gothic masterpi|P            |
|Urban park oasi|NYC          |
|Casino resorts,|LV           |
|Art Deco skyscr|NYC          |
|Suspension brid|SF           |
|Memorial to Pre|WDC          |
|Gaudí's unfinis|B            |
|Original Disney|A            |
|Disney World's |O            |
|Massive canyon |A            |
|Only remaining |C            |
|Tilted bell tow|P            |
|Opulent royal r|V            |
|Ancient citadel|A            |
+---------------+-------------+
only showing top 25 rows



In [42]:
df_feat = df_feat.withColumn(
    "Inicio_Campana",
    fun.to_date(fun.lit("2024-06-1"))
)

df_feat = df_feat.withColumn(
    "Fin_Campana",
    fun.date_add(fun.col("Inicio_Campana"),90)
)


df_feat.select("Inicio_Campana","Fin_Campana").show(25,truncate=False)

+--------------+-----------+
|Inicio_Campana|Fin_Campana|
+--------------+-----------+
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
|2024-06-01    |2024-08-30 |
+--------------+-----------+
only showing top 25 rows

